# Implementation of Using Convolution Filters with clustering techniques (DBSCAN) and Understanding of CNN via layer wise relevance propagation (or similar techniques) 

In [5]:
%time

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.preprocessing import normalize

plt.rcParams['figure.figsize'] = 5.0, 4.0

from pyts.transformation import GADF,GASF
from sklearn.preprocessing import normalize

import uproot
import torch


CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 26 µs


### Definitions

In [2]:
def standard(x,height,decay):
    y = height*np.exp(-x*decay)
    return y

def shifter(x,starts):
    L = int(starts*len(x))
    y = np.zeros(len(x))
    y[:L] = np.zeros(L) 
    y[L:] = x[:(len(x)-L)]
    return y

def comb_standard(x,second,height_1,decay_1,height_2,decay_2):
    L = int(second*len(x))
    y = np.zeros(len(x))
    y[:L] = standard(x[:L],height_1,decay_1)
    y[L:] = standard(x[:(len(x)-L)],height_2,decay_2)
    return y

def noiser(x,strength):
    y = x + np.random.normal(0,strength,len(x))
    return y

def noiser_long(x,strength):
    noise = np.random.normal(0,strength,len(x))
    y = x + np.cumsum(noise)*strength
    return y

def noiser_comb(x,sepfact,strength):
    L = int(sepfact*len(x))
    x[:L] = noiser(x[:L],strength)
    x[L:] = noiser_long(x[L:],strength)
    return x

def array_maker(entries):
    x = np.arange(0,1,1/4096)
    x = np.expand_dims(x,axis=0)
    x = np.tile(x,[entries,1])
    return x

def double(x,second,height_1,decay_1,height_2,decay_2,starts,sepfact=0.15,strength=0.02):
    y = comb_standard(x,second,height_1,decay_1,height_2,decay_2)
    y = shifter(y,starts)
    y = noiser_comb(y,sepfact,strength)
    return y

def single(x,height,decay,starts,sepfact=0.15,strength=0.02):
    y = standard(x,height,decay)
    y = shifter(y,starts)
    y = noiser_comb(y,sepfact,strength)
    return y

def event_creators_single(entries):
    x = array_maker(entries)
    for i in range(entries):
        w = np.random.normal(1,0.01)
        r = np.random.normal(5,2)
        s = np.random.normal(0.03,0.005)
        x[i] = single(x[i],w,r,s)
    return x

def event_creators_single_2(entries):
    x = array_maker(entries)
    for i in range(entries):
        w = np.random.normal(1,0.01)
        r = np.random.normal(10,2)
        s = np.random.normal(0.03,0.005)
        x[i] = single(x[i],w,r,s)
    return x

def event_creators_single_3(entries):
    x = array_maker(entries)
    for i in range(entries):
        w = np.random.normal(1,0.1)
        r = np.random.normal(4,1)
        s = np.random.normal(0.03,0.005)
        x[i] = single(x[i],w,r,s)
    return x

def event_creators_sharp(entries):
    x = array_maker(entries)
    for i in range(entries):
        w = np.random.normal(1,0.01)
        r = np.random.normal(100,20)
        s = np.random.normal(0.03,0.005)
        x[i] = single(x[i],w,r,s)    
    return x

def event_creators_double_equal(entries):
    x = array_maker(entries)
    for i in range(entries):
        w = np.random.normal(0.2,0.01)
        p = np.random.normal(1,0.01)
        q = np.random.normal(5,2)
        r = np.random.normal(1,0.01)
        s = np.random.normal(5,2)
        t = np.random.normal(0.03,0.005)
        x[i] = double(x[i],w,p,q,r,s,t)
    return x
    
def event_creators_double_unequal(entries):
    x = array_maker(entries)
    for i in range(entries):
        w = np.random.normal(0.2,0.01)
        p = np.random.normal(1,0.2)
        q = np.random.normal(50,10)
        r = np.random.normal(1,0.2)
        s = np.random.normal(50,10)
        t = np.random.normal(0.03,0.005)
        x[i] = double(x[i],w,p,q,r,s,t)
    return x

def event_creators_sharp_fat(entries):
    x = array_maker(entries)
    for i in range(entries):
        w = np.random.normal(0.015,0.001)
        p = np.random.normal(1,0.2)
        q = np.random.normal(100,20)
        r = np.random.normal(0.2,0.05)
        s = np.random.normal(5,2)
        t = np.random.normal(0.03,0.005)
        x[i] = double(x[i],w,p,q,r,s,t)
    return x

def label(q,k):
    x = np.zeros(len(q))
    for i in range(len(q)):
        x[i] = k
    return x

def sep(q,k,z):
    y = label(q,k)
    x1, x2 ,x3 = np.split(q,[int(len(q)*training_ratio),int(len(q)*(training_ratio+validation_ratio))])
    y1, y2 ,y3 = np.split(y,[int(len(q)*training_ratio),int(len(q)*(training_ratio+validation_ratio))])
    if z == 0:
        return x1, y1
    if z == 1:
        return x2, y2
    if z == 2:
        return x3, y3

def reader_pmtall(path):
    extra = np.arange(4096, 4480)
    tree = uproot.open(path)["tree"]
    pmtall = tree.array("PMTALL")
    pmtall = np.delete(pmtall, extra, axis=1)
    return pmtall

def reader(path,branch,number):
    tree = uproot.open(path)["tree"]
    column = tree.array(branch)
    column = column[:,number]
    return column

def reader_lone(path,branch):
    tree = uproot.open(path)["tree"]
    column = tree.array(branch)
    return column

def pmtall_pedestal(path):
    pedestal = reader(path,"Pedestal",0)
    pmtall = reader_pmtall(path)
    for i in range(len(pedestal)):
        pmtall[i] = -(pmtall[i]-pedestal[i])
    
    return pmtall

In [3]:
def comb(one,two,three,four,five,six,seven,eight,nine,ten,portion):
    one1,one2 = sep(one,1,portion)
    two1,two2 = sep(two,1,portion)
    three1,three2 = sep(three,1,portion)
    four1,four2 = sep(four,1,portion)
    five1,five2 = sep(five,1,portion)
    six1,six2 = sep(six,0,portion)
    seven1,seven2 = sep(seven,0,portion)
    eight1,eight2 = sep(eight,0,portion)
    nine1,nine2 = sep(nine,0,portion)
    ten1,ten2 = sep(ten,0,portion)

    z = np.concatenate((one1,two1,three1,four1,five1,six1,seven1,eight1,nine1,ten1),axis=0)
    y = np.concatenate((one2,two2,three2,four2,five2,six2,seven2,eight2,nine2,ten2),axis=0)
    return z, y

# Load data
build it using python class

In [9]:
%time
class Waveform():
    
    def __init__(self, path=None, no_classes=None):
        if path is None:
            raise ValueError("Insert file path!")
        if no_classes is None:
            raise ValueError("Number of classes?")
        
        # Load PMTALL(sum of waveform of CANDLES), removing last portion of data
        tree = uproot.open(path)["tree"]
        extra = np.arange(4096,4480)
        pmtall = tree.array("PMTALL")
        pmtall = np.delete(pmtall, extra, axis=1)
        pedestal = tree.array("Pedestal")
        pedestal_sum = pedestal[:,0]
        for i in range(len(pedestal_sum)):
            pmtall[i] = -(pmtall[i]-pedestal_sum[i])
#         number = 
        
        # random labelling(test purposes)
        self.waveform = pmtall
        self.label = np.random.randint(3,size=(len(pmtall),))
    
    def __len__(self):
        return self.waveform.shape[0]
    
    def __getitem__(self,idx):
        return (self.waveform[idx],self.label[idx])


CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.82 µs


In [3]:
no_classes = 3
dataset = Waveform(path="Run009-069-001.root", no_classes=no_classes)

In [6]:
BATCH_SIZE = 1000
from torch.utils.data import DataLoader
data_loader = DataLoader(dataset=dataset,
                         batch_size=BATCH_SIZE,
                         shuffle=True,
                         num_workers=3) 

# #Testing#
# for i in dataset:
#     print(len(i[0]),len(i[1]))
# for i in data_loader:
#     print(i[0].size())
#     print(i[1].size())
# print(len(dataset))


NameError: name 'dataset' is not defined

In [5]:
n_batches = int(len(dataset)/BATCH_SIZE)

In [6]:
n_batches

41

# Define CNN structure
using an autoencoder for self-training, taking encoder part or decoder part for features learning

In [64]:
torch.randn(10,10)


 0.1732 -0.7867  1.0059  0.7242 -0.6296 -1.8647  1.3830 -0.6172 -0.6004 -0.6850
-1.1482 -0.0940 -2.4120  1.2823  0.7159 -0.1950  1.1712 -0.2942 -0.1067 -0.0765
 1.1541  0.4790 -2.3716  1.7382  1.4308  1.0778  0.7979 -1.0894 -1.2398  0.3669
-1.1194  1.5223  0.0146 -1.9548  0.3726 -1.4361  1.4112 -0.9027  1.6328 -0.3982
-1.6377 -0.5356 -0.2465  0.2279  1.3486 -1.0220  0.1333  1.2906  1.4574  1.0001
-0.8595  0.5142 -0.2984 -1.0140  1.1616  0.0935 -0.7162  0.0328  1.7915  0.1084
 1.2633  1.6165  1.4476  0.6378 -2.6928  0.2502 -1.4982 -0.1744  0.2629  2.0208
-1.0208  1.5432 -0.3982 -0.9259  0.6875 -1.3311  0.5838 -0.0857 -1.2087 -1.2809
 0.4147 -0.5013  3.0467  1.7507  2.9731  1.2949  0.2020  0.6395 -0.0952  1.1100
 0.2361 -1.5246  0.2510  0.7203  0.4728  0.3298 -0.5078 -2.0981  1.1358  1.2134
[torch.FloatTensor of size 10x10]

In [7]:
from torch import nn
from torch.autograd import Variable

# Discriminator
class Classifier(nn.Module):
    def __init__(self, input_chn, no_classes, kernel_size, no_filters, padding, maxpool, batch_size):
        super(Classifier, self).__init__()
        self.input_chn = input_chn
        self.no_classes = no_classes
        self.kernel_size = kernel_size
        self.padding = padding
        self.no_filters = no_filters
        self.maxpool = maxpool
        self.batch_size = batch_size
        
        self.c1 = nn.Conv1d(input_chn ,no_filters ,kernel_size ,padding=padding )
        self.p1 = nn.MaxPool1d(maxpool ,padding )
        self.lr = nn.LeakyReLU(0.2)
        self.c2 = nn.Conv1d(no_filters, int(no_filters/2), kernel_size, padding=padding)
        self.p2 = nn.MaxPool1d(maxpool ,padding )       
        self.l1 = nn.Linear(4096,64)
        self.out = nn.Linear(64, no_classes)  
        self.sg = nn.Sigmoid()
        
    def forward(self, inputs):
        
        x = inputs.view(batch_size,1,-1)
        x = self.c1(inputs)
        x = self.lr(x)
        x = self.p1(x)
        x = self.c2(x)
        x = self.lr(x)
        x = self.p2(x)
        
        x = x.view(self.batch_size,-1)
        
        x = self.l1(x)
        x = self.lr(x)
        x = self.out(x)
        outputs = self.sg(x)
        return outputs

CNN = Classifier(1, no_classes=3, kernel_size=8, no_filters=32, padding=4, maxpool=2, batch_size=BATCH_SIZE) 

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(CNN.parameters(), lr=0.001, momentum=0.9)

print(CNN)
for parameter in CNN.parameters():
    print(parameter.size())

Classifier(
  (c1): Conv1d(1, 32, kernel_size=(8,), stride=(1,), padding=(4,))
  (p1): MaxPool1d(kernel_size=2, stride=4, padding=0, dilation=1, ceil_mode=False)
  (lr): LeakyReLU(0.2)
  (c2): Conv1d(32, 16, kernel_size=(8,), stride=(1,), padding=(4,))
  (p2): MaxPool1d(kernel_size=2, stride=4, padding=0, dilation=1, ceil_mode=False)
  (l1): Linear(in_features=4096, out_features=64, bias=True)
  (out): Linear(in_features=64, out_features=3, bias=True)
  (sg): Sigmoid()
)
torch.Size([32, 1, 8])
torch.Size([32])
torch.Size([16, 32, 8])
torch.Size([16])
torch.Size([64, 4096])
torch.Size([64])
torch.Size([3, 64])
torch.Size([3])


# training
transform to Torch.Variable

In [8]:
from torch.autograd import Variable

def to_var(x):
    # first move to GPU, if necessary
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x)

# Dummy code workspace

In [ ]:
output = Variable(torch.randn(10,120))
target = Variable(torch.FloatTensor(10).uniform_(0, 120).long())
print(torch.FloatTensor(10))
print(torch.FloatTensor(10).uniform_(0,1).long())
# print(output,target)

loss = criterion(output,target)
print(loss)

In [10]:
N_EPOCHS = 2

# allow for manual keyboard interrupt
try: 
    # loop through epochs
    for epoch in range(N_EPOCHS):
        for batch_number, (waveform,label) in enumerate(data_loader):
            
#             print("epoch=",epoch)
#             print(batch_number)
#             print(waveform.size(),label.size())
    
            batch_size = waveform.size()[0]
            training_data = to_var(waveform.view(batch_size,1,4096))
            target = to_var(label.view(batch_size,-1))
            
#             print(training_data,target)
            
            outputs = CNN(training_data)
#             print(outputs, target.view(-1).long())
            
            real_score = outputs
            loss = criterion(outputs, target.view(-1).long())
            loss.backward()
            optimizer.step()
#             print(loss.data[0])
        
            if (batch_number +1)%5 == 0:
                print("Epoch[%d/%d], Step[%d/%d], loss=%.4f, Mean Discriminator output=%.4f"
                      %(epoch,
                        N_EPOCHS,
                        batch_number+1,
                        n_batches,
                        loss.data[0],outputs.data.mean()))
            
            torch.save(CNN.state_dict(), "CNN.pkl")
        

except KeyboardInterrupt:
    print('Training ended early.')



Epoch[0/2], Step[5/41], loss=1.1120, Mean Discriminator output=0.6957
Epoch[0/2], Step[10/41], loss=1.1186, Mean Discriminator output=0.7564
Epoch[0/2], Step[15/41], loss=1.0974, Mean Discriminator output=0.8304
Epoch[0/2], Step[20/41], loss=1.0963, Mean Discriminator output=0.8430
Epoch[0/2], Step[25/41], loss=1.1000, Mean Discriminator output=0.8614
Epoch[0/2], Step[30/41], loss=1.1011, Mean Discriminator output=0.8904
Epoch[0/2], Step[35/41], loss=1.1005, Mean Discriminator output=0.9178
Epoch[0/2], Step[40/41], loss=1.0983, Mean Discriminator output=0.9508


RuntimeError: invalid argument 2: size '[1000 x -1]' is invalid for input with 1064960 elements at /Users/soumith/code/builder/wheel/pytorch-src/torch/lib/TH/THStorage.c:37

# Testing output of models

In [11]:
test = CNN(torch.randn((100,4096)))

## PLOT the test results



NameError: name 'batch_size' is not defined

# extracting filters/features

Use the protion of the Autoencoder transfer the trained weight to semi-half model of autoencoder.
Uses the output of the midlle layer of autoencoder and do a clustering on it.

The cluster number is a single vector(multi-class in keras?) representing the "output" of the group class.

# How to let this last layer of output automatically deduce itself.?
* save every sample output? how to make gives a supervised like methods output??

#Retrain the network but freeze all layer except for the last layer (clustering analysis layer). this can uses for LWRP studies, identify physical meaning of grouping. with it?

* understanding Neural Network, how to get layer-wise relevance propagation work for the last layer?

# Clustering techniques
most likely DBSCAN don't require to specify number of clusters however this can explode making difficult to use, the best option for now

# Layer Wise Relevance Propagation Or similiar techniques
The purpose of understanding CNN, see which portion of data gives more importance, mostly likely create a new CNN using trained weights from autoencoder with the final layers self-created based of the results of clustering. This can allow us to treate this CNN as supervised technique like however in reality as it is based on purely unsupervised techniques. This supervised like technique allows us to use technique like layer wise relevance propagation to understand the CNN giving us insight of the working of features learned by the cNN.